# Unlabeled Data Analysis

create twitter hashtag reader and reddit thread reader

pick sample hashtags and threads

## Twitter

In [1]:
import nest_asyncio
import twint
nest_asyncio.apply()

In [22]:
def get_tweets(search, limit=1000):
    
    c = twint.Config()
    c.Limit = limit
    c.Min_likes = 5
    c.Pandas = True
    c.Lang = 'en'
    c.Hide_output = True
    c.Search = search
    
    twint.run.Search(c)
    tweets = twint.storage.panda.Tweets_df
    tweets = tweets.loc[tweets['language']=='en']
    
    return tweets['tweet'].to_list()

In [26]:
model = load_model('final_model/model.h5')
model.load_weights('final_model/model_weights.h5')
vectorizer = load(open('final_model/vectorizer.pk', 'rb'))
    
def get_twitter_predictions(search, limit=1000):
    tweets = get_tweets(search, limit)
    X = process_data(tweets)
    
    X_processed = vectorizer.transform(process_data(X)).todense()
    y_pred = model.predict_classes(X_processed)
    df = pd.DataFrame(zip(X, y_pred.flatten()), columns=['tweet', 'positive'])
    
    return df

In [28]:
df = get_twitter_predictions('#StateofPlay')

C:\Users\Andrew\anaconda3\envs\capstone\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [29]:
df['positive'].value_counts(normalize=True)

1    0.958606
0    0.041394
Name: positive, dtype: float64

In [31]:
df['tweet'][df[df['positive']==True].index].tolist()[:5]

['trying to call upon my superpower from last year immediate announcement of final fantasy xvi to will an update into existence during today stateofplay',
 'im expecting absolutely nothing from playstation today stateofplay httpstcoizg9xexysi',
 'stateofplay is later today kid are yall ready for it i know i am game i reeeeally hope to see god of war ragnar k final fantasy xvi horizon forbidden west kena bridge of spirit holding out for an the order 1886 sequel too but i know it unlikely',
 'the newest playstation state of play is nearly here join me live a we watch and react to all the game being shown stateofplay stream link httpstco2wuyonqhs8 httpstcogdeavntnzz',
 'a little over 2 hour left make sure if you didnt check out my video to do so a little primer for stateofplay httpstcoz1swtvhlu8 httpstcosr7ca4ibqv']

In [32]:
df['tweet'][df[df['positive']==False].index].tolist()[:5]

['oh there a stateofplay today they really advertised that badly eh',
 '3 hour remain stateofplay',
 'live from 945pm gmt join ianhigton zoedels and aoifelockhart a we find out what sony ha up it sleeve in the latest stateofplay stream expect announcement deepdives and more httpstcoo27wbfokxi httpstcomxp8vmilts',
 'report playstation plus lineup reveal today either during stateofplay or a blog post immediately after the show',
 'i dont think we see horizonforbiddenwest in this stateofplay hfw is too big of a game to try and cram into a 30 min show save it for e3 to demo it properly ps5']

## Reddit

In [ ]:
from praw import Reddit
from config import reddit_api

In [ ]:
def get_comments(url):
    reddit = Reddit(client_id=reddit_api['client_id'],
                    client_secret=reddit_api['client_secret'],
                    user_agent=reddit_api['user_agent'])

    submissionId = url[url.find('comments'):].split('/')[1]
    submission = reddit.submission(submissionId)
    submission.comments.replace_more(limit=None)
    comments = []
    for comment in submission.comments:
        comments.append(comment.body)
        
    return comments

In [ ]:
X = get_comments('https://www.reddit.com/r/Games/comments/ls33hs/activision_warns_a_standard_500gb_ps4_may_no/')

load best processing and model pickles

## Processing Data

In [24]:
from keras.models import load_model
import pandas as pd
from pickle import load
import preprocessing

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def process_data(X):
    X_pre = list(map(preprocessing.remove_markdown, X))
    X_pre = list(map(preprocessing.remove_punctuation, X_pre))
    X_pre = list(map(preprocessing.tokenize, X_pre))
    X_pre = list(map(preprocessing.lemmatize, X_pre))
    X_join = [' '.join(x) for x in X_pre]
    
    return X_join

In [ ]:
model = load_model('final_model/model.h5')
model.load_weights('final_model/model_weights.h5')
vectorizer = load(open('final_model/vectorizer.pk', 'rb'))
    
def get_reddit_predictions(url):
    comments = get_comments(url)
    X = process_data(comments)
    
    X_processed = vectorizer.transform(process_data(X)).todense()
    y_pred = model.predict_classes(X_processed)
    df = pd.DataFrame(zip(X, y_pred.flatten()), columns=['comment', 'positive'])
    
    return df

sample reddit thread ideas:
- https://www.reddit.com/r/nintendo/comments/lm6obv/project_triangle_strategy_announcement_trailer/
- https://www.reddit.com/r/nintendo/comments/lru33p/the_animal_crossing_new_horizons_free_update_is/
- https://www.reddit.com/r/nintendo/comments/lqlrqg/tony_hawks_pro_skater_1_2_coming_to_nintendo/
- https://www.reddit.com/r/Games/comments/ls6qlh/bravery_default_ii_review_thread/

In [ ]:
df = get_reddit_predictions('https://www.reddit.com/r/Games/comments/lm6ns2/project_triangle_strategy_announcement_trailer/')

In [ ]:
df['positive'].value_counts(normalize=True)

In [ ]:
df['comment'][df[df['positive']==True].index].tolist()[:5]

In [ ]:
df['comment'][df[df['positive']==False].index].tolist()[:5]

run data through processing and model

output class percentages and numbers

output unlabeled topic analysis

CREATE FRONTEND!!!